# Title Simple Stock Price Prediction Model
Here I will be implementing a simple MLP neural network for stock price prediction.  My GitHub profile is htjames0 and all code and files can be found in the repository. 

In [2]:
import tensorflow as tf
import pandas as pd 
import matplotlib.pyplot as plt

Init Plugin
Init Graph Optimizer
Init Kernel


In [6]:
data = pd.read_csv('AMD.csv')
#features will be Open, High, Low, and Volume
#label will be Close

            Date        Open        High         Low       Close   Adj Close  \
1399  2021-07-26   92.010002   92.750000   91.120003   91.820000   91.820000   
1400  2021-07-27   92.940002   94.099998   89.099998   91.029999   91.029999   
1401  2021-07-28   93.440002   98.709999   89.650002   97.930000   97.930000   
1402  2021-07-29   96.580002  105.739998   96.580002  102.949997  102.949997   
1403  2021-07-30  101.599998  106.970001  101.379997  106.190002  106.190002   

         Volume  
1399   27668500  
1400   69427000  
1401  140561000  
1402  164091800  
1403  125567000  


In [7]:
def MLPmodel(features, step_size, hidden_neurons=5, act='relu', loss_fxn='mean_squared_error'):
    
    #method inputs
        #features - feature data used to get the number of input layer nodes
        #step-size - for optimizer
        #hidden_neurons - number of neurons in hidden layer, default of five
        #act - activation function, default ReLu function
        #loss_fxn - loss function, default as mean squared error
    
    #defining model
    model = tf.keras.models.Sequential()
   
    #layers
    model.add(tf.keras.layers.Dense(units=hidden_neurons,
                                    input_shape=len(features.columns),
                                    activation='relu')
                                   )
    model.add(tf.keras.layers.Dense(units = 1,
                                    activation=act)
                                   )
    
    #optimizer
    opt = tf.keras.optimizers.Adam(learn_rate=step_size)
    
    #compile - loss fxn MSE
    model.compile(optimizer=opt,
                  loss=loss_fxn,
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
   
    return model 

In [12]:
#train function
def train_model(model, feature, label, epochs, batch_size):

    #feeding features and labels to model, model iterates epoch number of times
    #using batch_size number of data points per iteration
    history = model.fit(x=feature,
                        y=label,
                        batch_size=batch_size,
                        epochs=epochs)

    #getting weights and bias
    trained_weight = model.get_weights()[0]
    trained_bias = model.get_bias()[1]

    #getting historical data of model for each epoch
    epochs = history.epoch
    hist = pd.DateFram(history.history)

    #getting rmse for each epoch
    rmse = hist["root_mean_squared_error"]

    return trained_weight, trained_bias, epochs, rmse

In [13]:
#predict function
